In [54]:
import warnings
warnings.filterwarnings("ignore")

import os
from langchain_community.chat_models.azure_openai import AzureChatOpenAI
from langchain_community.embeddings import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import Dict
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch


In [3]:
os.environ["OPENAI_API_KEY"] = "b7d5aa82d15a4b99a1c730f681ec2bbc"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://hmh-digitalhub-azure-openai.openai.azure.com/"
os.environ["CHAT_MODEL"] = "gpt-35-turbo"
os.environ["CHAT_MODEL_DEPLOYMENT_NAME"] = "gpt-35-turbo"
os.environ["EMBEDDINGS_MODEL"] = "text-embedding-ada-002"
os.environ["EMBEDDINGS_MODEL_DEPLOYMENT_NAME"] = "text-embedding-ada-002"

llm = AzureChatOpenAI(  
            model_name ="gpt-35-turbo",
            deployment_name= "gpt-35-turbo",
            temperature=0,
            openai_api_version ="2023-07-01-preview",
            openai_api_key="b7d5aa82d15a4b99a1c730f681ec2bbc",
            openai_api_base="https://hmh-digitalhub-azure-openai.openai.azure.com/"
        )

embeddings = AzureOpenAIEmbeddings()

c:\Users\shyam\.conda\envs\lc-prac\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [5]:
llm.invoke(
    [
        HumanMessage(
            content= "Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content="J'adore la programmation.", response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 19, 'total_tokens': 27}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff5ed168-0aa0-4911-915f-9598cd8257d6-0')

In [6]:
llm.invoke([HumanMessage(content= 'What did you just said?')])

AIMessage(content='I apologize if there was a misunderstanding. Can you please clarify what you are referring to?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-3976d0f0-df31-4c85-810b-80fc910d9548-0')

In [9]:
llm.invoke(
    [
        HumanMessage(content= "Translate this sentence from English to French: I love programming."),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content= 'What did you just said?')
    ]
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 41, 'total_tokens': 62}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-372e7b4f-f9d4-4520-973b-b516c97aa6c8-0')

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [14]:
chain.invoke(
    {
        'messages': [
            HumanMessage(content= "Translate this sentence from English to French: I love programming."),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content= 'What did you just said?')
        ]
    }
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 61, 'total_tokens': 82}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-09185170-8c95-4b5b-99bd-0e569a6c3a54-0')

In [17]:
chat_history= ChatMessageHistory()

chat_history.add_user_message('Hi!')
chat_history.add_ai_message('Whats up?')
chat_history.messages

[HumanMessage(content='Hi!'), AIMessage(content='Whats up?')]

In [23]:
chat_history.add_user_message("Translate this sentence from English to French: I love programming.")
response= chain.invoke({'messages': chat_history.messages})
response

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 100, 'total_tokens': 120}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-52b6ba3c-81fb-4dad-97c6-164f42d683cb-0')

In [26]:
chat_history.add_ai_message(response)
chat_history.add_user_message('What did you just said?')

chain.invoke({'messages': chat_history.messages})

AIMessage(content='I said, "The translation of \'I love programming\' in French is \'J\'adore la programmation.\'"', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 202, 'total_tokens': 226}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-30e33e12-278c-48b1-ac86-7722300029db-0')

In [28]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [31]:
len(data)

1

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [36]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding= embeddings)

In [41]:
retriever= vectorstore.as_retriever(k= 4)
docs= retriever.invoke('how can langsmith help with testing?')
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearc

In [44]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        'system',
        "Answer the user's question based on the given context.\n\n {context}",
        MessagesPlaceholder(variable_name= 'messages'),
    ]
)
document_chain= create_stuff_documents_chain(llm, qa_prompt)

In [46]:
chat_history= ChatMessageHistory()
chat_history.add_user_message('how can langsmith help with testing?')

document_chain.invoke(
    {'messages': chat_history.messages,
     'context': docs}
)

"LangSmith can help with testing by allowing you to closely monitor and evaluate your application, so you can ship quickly and with confidence. It provides a platform for building production-grade LLM applications and offers features for tracing and evaluating your application's performance. Additionally, LangSmith does not require the use of LangChain, making it a versatile tool for testing and monitoring applications."

In [48]:
#extracting the last human message from the passed message history
def parse_retriever_input(params: Dict):
    return params['messages'][-1].content

retrieval_chain= RunnablePassthrough.assign(
    context= parse_retriever_input | retriever
).assign(
    answer= document_chain
)

In [49]:
response= retrieval_chain.invoke(
    {'messages': chat_history.messages}
)
response

{'messages': [HumanMessage(content='how can langsmith help with testing?')],
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
  Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith wo

In [52]:
chat_history.add_ai_message(response['answer'])
chat_history.add_user_message('tell me more about that!')

retrieval_chain.invoke(
    {'messages': chat_history.messages}
)

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content="LangSmith can help with testing by allowing you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. It provides a platform for building production-grade LLM applications and offers features for tracing and evaluating your application's performance. Additionally, LangSmith does not require the use of LangChain, making it a versatile tool for testing and monitoring applications."),
  AIMessage(content="LangSmith can help with testing by allowing you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. It provides a platform for building production-grade LLM applications and offers features for tracing and evaluating your application's performance. Additionally, LangSmith does not require the use of LangChain, making it a versatile tool for testing and monitoring applications."),
  HumanMessage(conte

In [53]:
retriever_chain_with_only_answer= (
    RunnablePassthrough.assign(
        context= parse_retriever_input | retriever
    ) | document_chain
)

retriever_chain_with_only_answer.invoke(
    {'messages': chat_history.messages}
)

"LangSmith provides a comprehensive set of tools for tracing and evaluating the performance of your application. It allows you to monitor the behavior of your application in real-time and gather valuable insights into its performance. This can help you identify and address any issues or bottlenecks, ultimately leading to more reliable and efficient software development. Additionally, LangSmith's integration with OpenAI and its ability to trace and evaluate LLM (Large Language Model) calls make it a powerful tool for testing and monitoring applications that rely on natural language processing."

- One of the problem is during the first question asked, the retriever is able to retrieve relevant chunks of data.
- But during the follow up question the retriever doesn't have the previous context to retrieve the relevant info
- So we need to transform our query for better retrieval

In [55]:
# #RunnableBranch selects which branch to run based on a condition.
# branch = RunnableBranch(
#     (lambda x: isinstance(x, str), lambda x: x.upper()),
#     (lambda x: isinstance(x, int), lambda x: x + 1),
#     (lambda x: isinstance(x, float), lambda x: x * 2),
#     lambda x: "goodbye",
# )

# branch.invoke("hello") # "HELLO"
# branch.invoke(None) # "goodbye"

In [59]:
query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | llm | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [61]:
document_chain = create_stuff_documents_chain(llm, qa_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

chat_history = ChatMessageHistory()

In [62]:
chat_history.add_user_message("how can langsmith help with testing?")

response = conversational_retrieval_chain.invoke(
    {"messages": chat_history.messages},
)

chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content="LangSmith can help with testing by allowing you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. It provides a platform for building production-grade LLM applications and offers features for tracing and evaluating your application's performance. Additionally, LangSmith does not require the use of LangChain, making it a versatile tool for testing and monitoring applications.")],
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started 

In [63]:
chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": chat_history.messages}
)

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content="LangSmith can help with testing by allowing you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. It provides a platform for building production-grade LLM applications and offers features for tracing and evaluating your application's performance. Additionally, LangSmith does not require the use of LangChain, making it a versatile tool for testing and monitoring applications."),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its 